In [1]:
# Require libraries
import matplotlib.pyplot as plt
import os
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import numpy as np
import glob
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.signal import savgol_filter
from tsmoothie.smoother import LowessSmoother


In [2]:
# Function for pixel to degree conversion

# for stimulus
def px_deg_sti(data):
    time_sti = []
    pos_sti = []
    for y in data[1]:
        new_y = round((y - 960)/49.6,4) # horizontal center:960x540 pixel : 1 deg = 49.6 pixel
        pos_sti.append(new_y)

    for x in range(1,len(pos_sti)+1):
        new_x = round((1/60)*x,4) # for monitor with screen refresh rate( 60 Hz)
        time_sti.append(new_x)
    return time_sti,pos_sti
    
# for eye
def px_deg_eye(data,dominant_eye):
    time_eye = []
    pos_eye = []
    if dominant_eye == 'right':
        for y in data['GazePointRightX (ADCSpx)']:
            new_y = round((y - 960)/49.6,4) # horizontal center:960x540 pixel : 1 deg = 49.6 pixel
            pos_eye.append(new_y)
    elif dominant_eye == 'left':
        for y in data['GazePointLeftX (ADCSpx)']:
            new_y = round((y - 960)/49.6,4) # horizontal center:960x540 pixel : 1 deg = 49.6 pixel
            pos_eye.append(new_y)
    for x in range(1,len(pos_eye)+1):
        new_x = round((1/300)*x,4) # for tobii's eye tracker with 300 Hz sampling rate
        time_eye.append(new_x)
    return time_eye,pos_eye

In [3]:
# Create path for healthy control(HC)
path = os.getcwd()
dirname = os.path.dirname(path)
option = ['URLStart','URLEnd'] # This is the values in the Studio event column where stimuli start and end events from dataset
directory_hc  = dirname+"/eye_movements_sp/subjects/hc"
# s_path = dirname+"/result/4_dps/s_hc" # single gasf path for hc
# c_path = dirname+"/result/4_dps/c_hc" # combine gasf path for pd
files_hc = list(glob.iglob(directory_hc+'/[!.]*')) # skip unwanted file such as .DS_Store file
right = 'right'
left = 'left'

In [4]:
# # Remove outlier
# def remove_outlier_IQR(df, column):
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_limit = Q1 - 1.5 * IQR
#     upper_limit = Q3 + 1.5 * IQR
#     mask = (df[column] >= lower_limit) & (df[column] <= upper_limit)
#     return df.loc[mask]

In [7]:
# # Blink detection and interpolation
# def blink_nan(series,i):
#     df = pd.DataFrame(series)
#     df['new'] = ((df[i].notnull() != df[i].shift().notnull()).cumsum())
#     df['ones'] =1
#     mask_zeros = (df.groupby('new')['ones'].transform('count') == 23 ) | df[i].notnull()
#     #mask_zeros = (df.groupby('new')['ones'].transform('count') > 23 ) & (df.groupby('new')['ones'].transform('count') < 50) | df[i].notnull()
#     blink_series = series.fillna(0).where(mask_zeros)
#     interpolate_series = pd.Series(blink_series)
#     interpolate_df = pd.DataFrame(interpolate_series)
#     interpolate_df = interpolate_df.interpolate().ffill().bfill()
#     return interpolate_df

In [8]:
directory_sti  = dirname+"/data/stimulus"
files_sti = glob.iglob(directory_sti+'/[!.]*')
for file_sti in files_sti: 
    # for stimulus (speed = 4deg/s, duration = 23 sec)
    data_sti = pd.read_csv(file_sti,sep='\s+',header=None)
    one_deg_time_sti , one_deg_pos_sti =  px_deg_sti(data_sti)
    stimulus = pd.DataFrame({'x':one_deg_time_sti,'y':one_deg_pos_sti})
    stimulus.to_csv(str(Path(file_sti).stem)+'_sti.csv',index=False)

In [237]:
# one degree
def OneCsvHC(speed):
    frames = []
    for file_hc in files_hc:  
        print('File '+ str(os.path.basename(file_hc))+ ' is started.') 
        # Data preporcessing
        # Interpolate the NaN values with linear
        data_hc  = pd.read_excel(file_hc+'/'+speed)#.interpolate()
        t = data_hc[data_hc['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
        
        if right in str(os.path.basename(file_hc)):
            # First Trial
            data_eye_hc_first = data_hc[t[0]+902:t[0]+27904] # first trial index
            #data_eye_hc_first  = remove_outlier_IQR(data_eye_hc_first,'GazePointRightX (ADCSpx)')
            #data_eye_hc_first  = data_eye_hc_first.to_frame()
            #data_eye_hc_first  =  blink_nan(data_eye_hc_first['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_hc_first  = data_eye_hc_first.to_frame()
            one_deg_time_eye , one_deg_pos_eye =  px_deg_eye(data_eye_hc_first,'right')
            
            #Remove left and right
            filename = os.path.basename(file_hc)
            filename = filename.replace('_right','').replace('_left','')
            one_deg_first = pd.DataFrame({'x2_'+str(filename):one_deg_time_eye,
            'y2_'+str(filename):one_deg_pos_eye})


            # Second Trial
            data_eye_hc_second = data_hc[t[2]+902:t[2]+27904]
            #data_eye_hc_second = remove_outlier_IQR(data_eye_hc_second,'GazePointRightX (ADCSpx)')
            #data_eye_hc_second  = data_eye_hc_second.to_frame()
            #data_eye_hc_second  =  blink_nan(data_eye_hc_second['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_hc_second  = data_eye_hc_second.to_frame()
            one_deg_time_eye_second , one_deg_pos_eye_second =  px_deg_eye(data_eye_hc_second,'right')
            
            #Remove left and right
            filename = os.path.basename(file_hc)
            filename = filename.replace('_right','').replace('_left','')
            one_deg_second = pd.DataFrame({'x2_'+str(filename):one_deg_time_eye_second,
            'y2_'+str(filename):one_deg_pos_eye_second})

            df_subject = pd.concat([one_deg_first,one_deg_second],axis=1)

            frames.append(df_subject)
            print('File '+ str(filename)+ ' is done.')
        else:
            # First Trial
            data_eye_hc_first = data_hc[t[0]+902:t[0]+27904] # first trial index
            #data_eye_hc_first  = remove_outlier_IQR(data_eye_hc_first,'GazePointLeftX (ADCSpx)')
            #data_eye_hc_first  = data_eye_hc_first.to_frame()
            #data_eye_hc_first  =  blink_nan(data_eye_hc_first['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_hc_first  = data_eye_hc_first.to_frame()
            one_deg_time_eye , one_deg_pos_eye =  px_deg_eye(data_eye_hc_first,'left')
            
            #Remove left and right
            filename = os.path.basename(file_hc)
            filename = filename.replace('_right','').replace('_left','')
            one_deg_first = pd.DataFrame({'x2_'+str(filename):one_deg_time_eye,
            'y2_'+str(filename):one_deg_pos_eye})


            # Second Trial
            data_eye_hc_second = data_hc[t[2]+902:t[2]+27904]
            #data_eye_hc_second = remove_outlier_IQR(data_eye_hc_second,'GazePointLeftX (ADCSpx)')
            #data_eye_hc_second  = data_eye_hc_second.to_frame()
            #data_eye_hc_second  =  blink_nan(data_eye_hc_second['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_hc_second  = data_eye_hc_second.to_frame()
            one_deg_time_eye_second , one_deg_pos_eye_second =  px_deg_eye(data_eye_hc_second,'left')
            
            #Remove left and right
            filename = os.path.basename(file_hc)
            filename = filename.replace('_right','').replace('_left','')
            one_deg_second = pd.DataFrame({'x2_'+str(filename):one_deg_time_eye_second,
            'y2_'+str(filename):one_deg_pos_eye_second})

            df_subject = pd.concat([one_deg_first,one_deg_second],axis=1)

            frames.append(df_subject)

            print('File '+ str(filename)+ ' is done.')


    one_deg = pd.concat(frames,axis=1)
   
    one_deg.to_csv('one_degHC.csv',index=False)

In [239]:
OneCsvHC('1_dps.xlsx')

In [226]:
# two degree
def TwoCsvHC(speed):
    frames = []
    for file_hc in files_hc:  
        # Data preporcessing
        # Interpolate the NaN values with linear
        data_hc  = pd.read_excel(file_hc+'/'+speed)#.interpolate()
        t = data_hc[data_hc['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
        
        if right in str(os.path.basename(file_hc)):
            print('File '+ str(os.path.basename(file_hc))+ ' is started.') 
            # First Trial
            data_eye_hc_first = data_hc[t[0]+902:t[0]+14404] # first trial index
            #data_eye_hc_first  = remove_outlier_IQR(data_eye_hc_first,'GazePointRightX (ADCSpx)')
            #data_eye_hc_first  = data_eye_hc_first.to_frame()
            #data_eye_hc_first  =  blink_nan(data_eye_hc_first['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_hc_first  = data_eye_hc_first.to_frame()
            two_deg_time_eye , two_deg_pos_eye =  px_deg_eye(data_eye_hc_first,'right')
            
            #Remove left and right
            filename = os.path.basename(file_hc)
            filename = filename.replace('_right','').replace('_left','')
            
            #Remove left and right
            filename = os.path.basename(file_hc)
            filename = filename.replace('_right','').replace('_left','')
            two_deg_first = pd.DataFrame({'x2_'+str(filename):two_deg_time_eye,
            'y2_'+str(filename):two_deg_pos_eye})


            # Second Trial
            data_eye_hc_second = data_hc[t[2]+902:t[2]+14404]
            #data_eye_hc_second= remove_outlier_IQR(data_eye_hc_second,'GazePointRightX (ADCSpx)')
            #data_eye_hc_second  = data_eye_hc_second.to_frame()
            #data_eye_hc_second  =  blink_nan(data_eye_hc_second['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_hc_second  = data_eye_hc_second.to_frame()
            two_deg_time_eye_second , two_deg_pos_eye_second =  px_deg_eye(data_eye_hc_second,'right')
            
            #Remove left and right
            filename = os.path.basename(file_hc)
            filename = filename.replace('_right','').replace('_left','')
            two_deg_second = pd.DataFrame({'x2_'+str(filename):two_deg_time_eye_second,
            'y2_'+str(filename):two_deg_pos_eye_second})

            df_subject = pd.concat([two_deg_first,two_deg_second],axis=1)

            frames.append(df_subject)

            print('File '+ str(filename)+ ' is done.')
        else:
            # First Trial
            data_eye_hc_first = data_hc[t[0]+902:t[0]+14404] # first trial index
            #data_eye_hc_first = remove_outlier_IQR(data_eye_hc_first,'GazePointLeftX (ADCSpx)')
            #data_eye_hc_first  = data_eye_hc_first.to_frame()
            #data_eye_hc_first  =  blink_nan(data_eye_hc_first['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_hc_first  = data_eye_hc_first.to_frame()
            two_deg_time_eye , two_deg_pos_eye =  px_deg_eye(data_eye_hc_first,'left')
            
            #Remove left and right
            filename = os.path.basename(file_hc)
            filename = filename.replace('_right','').replace('_left','')
            two_deg_first = pd.DataFrame({'x2_'+str(filename):two_deg_time_eye,
            'y2_'+str(filename):two_deg_pos_eye})


            # Second Trial
            data_eye_hc_second = data_hc[t[2]+902:t[2]+14404]
            #data_eye_hc_second = remove_outlier_IQR(data_eye_hc_second,'GazePointLeftX (ADCSpx)')
            #data_eye_hc_second  = data_eye_hc_second.to_frame()
            #data_eye_hc_second  =  blink_nan(data_eye_hc_second['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_hc_second  = data_eye_hc_second.to_frame()
            two_deg_time_eye_second , two_deg_pos_eye_second =  px_deg_eye(data_eye_hc_second,'left')

            #Remove left and right
            filename = os.path.basename(file_hc)
            filename = filename.replace('_right','').replace('_left','')
            two_deg_second = pd.DataFrame({'x2_'+str(filename):two_deg_time_eye_second,
            'y2_'+str(filename):two_deg_pos_eye_second})

            df_subject = pd.concat([two_deg_first,two_deg_second],axis=1)

            frames.append(df_subject)

            print('File '+ str(filename)+ ' is done.')


    two_deg = pd.concat(frames,axis=1)
   
    two_deg.to_csv('two_degHC.csv',index=False)

In [228]:
TwoCsvHC('2_dps.xlsx')

In [23]:
# Four degree
def FourCsvHC(speed):
    frames = []
    processed_files_count = 0 
    for file_hc in files_hc:  
        try:
            print('File '+ str(os.path.basename(file_hc))+ ' is started.') 
            # Data preporcessing
            # Interpolate the NaN values with linear
            data_hc  = pd.read_excel(file_hc+'/'+speed)#.interpolate()
            t = data_hc[data_hc['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
            
            if right in str(os.path.basename(file_hc)):
                # First Trial
                data_eye_hc_first = data_hc[t[0]+902:t[0]+7804] # first trial index
                #data_eye_hc_first  = remove_outlier_IQR(data_eye_hc_first,'GazePointRightX (ADCSpx)')
                #outlier_first  = outlier_first.to_frame()
                #data_eye_hc_first  =  blink_nan(data_eye_hc_first['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
                #data_eye_hc_first  = data_eye_hc_first.to_frame()
                four_deg_time_eye , four_deg_pos_eye =  px_deg_eye(data_eye_hc_first,'right')
                
                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                four_deg_first = pd.DataFrame({'x2_'+str(filename):four_deg_time_eye,
                'y2_'+str(filename):four_deg_pos_eye})


                # Second Trial
                data_eye_hc_second = data_hc[t[2]+902:t[2]+7804]
                #data_eye_hc_second = remove_outlier_IQR(data_eye_hc_second,'GazePointRightX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                #data_eye_hc_second  =  blink_nan(data_eye_hc_second['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                four_deg_time_eye_second , four_deg_pos_eye_second =  px_deg_eye(data_eye_hc_second,'right')
                
                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                four_deg_second = pd.DataFrame({'x2_'+str(filename):four_deg_time_eye_second,
                'y2_'+str(filename):four_deg_pos_eye_second})

                df_subject = pd.concat([four_deg_first,four_deg_second],axis=1)

                frames.append(df_subject)
                processed_files_count += 1 
                print('File '+ str(filename)+ ' is done.')
            else:
                print('File '+ str(os.path.basename(file_hc))+ ' is started.')
                # First Trial
                data_eye_hc_first = data_hc[t[0]+902:t[0]+7804] # first trial index
                #data_eye_hc_first  = remove_outlier_IQR(data_eye_hc_first,'GazePointLeftX (ADCSpx)')
                #data_eye_hc_first  = data_eye_hc_first.to_frame()
                #data_eye_hc_first  =  blink_nan(data_eye_hc_first['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
                #data_eye_hc_first  = data_eye_hc_first.to_frame()
                four_deg_time_eye , four_deg_pos_eye =  px_deg_eye(data_eye_hc_first,'left')

                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                four_deg_first = pd.DataFrame({'x2_'+str(filename):four_deg_time_eye,
                'y2_'+str(filename):four_deg_pos_eye})


                # Second Trial
                data_eye_hc_second = data_hc[t[2]+902:t[2]+7804]
                #print("Before remove_outlier_IQR:", data_eye_hc_second.columns)
                #data_eye_hc_second = remove_outlier_IQR(data_eye_hc_second,'GazePointLeftX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                # Add this line to print the columns
                #print("After remove_outlier_IQR:", data_eye_hc_second.name)
                #data_eye_hc_second  =  blink_nan(data_eye_hc_second['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                four_deg_time_eye_second , four_deg_pos_eye_second =  px_deg_eye(data_eye_hc_second,'left')
                
                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                four_deg_second = pd.DataFrame({'x2_'+str(filename):four_deg_time_eye_second,
                'y2_'+str(filename):four_deg_pos_eye_second})

                df_subject = pd.concat([four_deg_first,four_deg_second],axis=1)

                frames.append(df_subject)
                processed_files_count += 1 
                print('File '+ str(filename)+ ' is done.')
        except Exception as e:
            print(f"Error processing file {os.path.basename(file_hc)}: {e}")
    print("Total number of processed files:", processed_files_count)  # Print the counter value after the loop


    four_deg = pd.concat(frames,axis=1)
   
    four_deg.to_csv('four_degHC.csv',index=False)

In [24]:
FourCsvHC('4_dps.xlsx')

File PD009_right is started.


In [21]:
# six degree
def SixCsvHC(speed):
    frames = []
    processed_files_count = 0 
    for file_hc in files_hc:
        try:   
            # Data preporcessing
            # Interpolate the NaN values with linear
            data_hc  = pd.read_excel(file_hc+'/'+speed)#.interpolate()
            t = data_hc[data_hc['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
            
            if right in str(os.path.basename(file_hc)):
                print('File '+ str(os.path.basename(file_hc))+ ' is started.') 
                # First Trial
                data_eye_hc_first = data_hc[t[0]+902:t[0]+5404] # first trial index
                #data_eye_hc_first  = remove_outlier_IQR(data_eye_hc_first,'GazePointRightX (ADCSpx)')
                #data_eye_hc_first  = data_eye_hc_first.to_frame()
                #data_eye_hc_first  =  blink_nan(data_eye_hc_first['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
                #data_eye_hc_first  = data_eye_hc_first.to_frame()
                six_deg_time_eye , six_deg_pos_eye =  px_deg_eye(data_eye_hc_first,'right')
                
                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                six_deg_first = pd.DataFrame({'x2_'+str(filename):six_deg_time_eye,
                'y2_'+str(filename):six_deg_pos_eye})



                # Second Trial
                data_eye_hc_second = data_hc[t[2]+902:t[2]+5404]
                #data_eye_hc_second = remove_outlier_IQR(data_eye_hc_second,'GazePointRightX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                #data_eye_hc_second  =  blink_nan(data_eye_hc_second['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                six_deg_time_eye_second , six_deg_pos_eye_second =  px_deg_eye(data_eye_hc_second,'right')
                
                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                six_deg_second = pd.DataFrame({'x2_'+str(filename):six_deg_time_eye_second,
                'y2_'+str(filename):six_deg_pos_eye_second})

                df_subject = pd.concat([six_deg_first,six_deg_second],axis=1)

                frames.append(df_subject)
                processed_files_count += 1  # Increment the counter for each successfully processed file

                print('File '+ str(filename)+ ' is done.')
            else:
                print('File '+ str(os.path.basename(file_hc))+ ' is started.')
                # First Trial
                data_eye_hc_first = data_hc[t[0]+902:t[0]+5404] # first trial index
                #data_eye_hc_first  = remove_outlier_IQR(data_eye_hc_first,'GazePointLeftX (ADCSpx)')
                #data_eye_hc_first  = data_eye_hc_first.to_frame()
                #data_eye_hc_first  =  blink_nan(data_eye_hc_first['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
                #data_eye_hc_first  = data_eye_hc_first.to_frame()
                six_deg_time_eye , six_deg_pos_eye =  px_deg_eye(data_eye_hc_first,'left')
                
                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                six_deg_first = pd.DataFrame({'x2_'+str(filename):six_deg_time_eye,
                'y2_'+str(filename):six_deg_pos_eye})



                # Second Trial
                data_eye_hc_second = data_hc[t[2]+902:t[2]+5404]
                #data_eye_hc_second = remove_outlier_IQR(data_eye_hc_second,'GazePointLeftX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                #data_eye_hc_second  =  blink_nan(data_eye_hc_second['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                six_deg_time_eye_second , six_deg_pos_eye_second =  px_deg_eye(data_eye_hc_second,'left')

                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                six_deg_second = pd.DataFrame({'x2_'+str(filename):six_deg_time_eye_second,
                'y2_'+str(filename):six_deg_pos_eye_second})

                df_subject = pd.concat([six_deg_first,six_deg_second],axis=1)

                frames.append(df_subject)
                processed_files_count += 1  # Increment the counter for each successfully processed file
                print('File '+ str(filename)+ ' is done.')
        except Exception as e:
            print(f"Error processing file {os.path.basename(file_hc)}: {e}")
    print("Total number of processed files:", processed_files_count)  # Print the counter value after the loop


    six_deg = pd.concat(frames,axis=1)
   
    six_deg.to_csv('six_degHC.csv',index=False)

In [22]:
SixCsvHC('6_dps.xlsx')

File PD009_right is started.
File PD009 is done.
File PDN015_right is started.
File PDN015 is done.
File PDN017_right is started.
File PDN017 is done.
File PD012_left is started.
File PD012 is done.
File PD004_left is started.
File PD004 is done.
File PD013_right is started.
File PD013 is done.
File PD002_left is started.
File PD002 is done.
File PD008_right is started.
File PD008 is done.
File PD011_right is started.
File PD011 is done.
File PD003_left is started.
File PD003 is done.
File PD026_right is started.
File PD026 is done.
File PD024_right is started.
File PD024 is done.
File PD020_right is started.
File PD020 is done.
File PDN018_right is started.
File PDN018 is done.
File PD022_right is started.
File PD022 is done.
File PD001_right is started.
File PD001 is done.
File PD006_left is started.
File PD006 is done.
File PD010_left is started.
File PD010 is done.
File PD007_right is started.
File PD007 is done.
File PD023_right is started.
File PD023 is done.
File PDN019_right is

In [16]:
# eight degree
def EightCsvHC(speed):
    frames = []
    processed_files_count = 0  # Initialize the counter here
    for file_hc in files_hc: 
        try: 
            # Data preporcessing
            # Interpolate the NaN values with linear
            data_hc  = pd.read_excel(file_hc+'/'+speed)#.interpolate()
            t = data_hc[data_hc['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
            
            if right in str(os.path.basename(file_hc)):
                print('File '+ str(os.path.basename(file_hc))+ ' is started.') 
                # First Trial
                data_eye_hc_first = data_hc[t[0]+902:t[0]+4204] # first trial index
                #data_eye_hc_first  = remove_outlier_IQR(data_eye_hc_first,'GazePointRightX (ADCSpx)')
                #data_eye_hc_first  = data_eye_hc_first.to_frame()
                #data_eye_hc_first  =  blink_nan(data_eye_hc_first['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
                #data_eye_hc_first  = data_eye_hc_first.to_frame()
                eight_deg_time_eye , eight_deg_pos_eye =  px_deg_eye(data_eye_hc_first,'right')

                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                eight_deg_first = pd.DataFrame({'x2_'+str(filename):eight_deg_time_eye,
                'y2_'+str(filename):eight_deg_pos_eye})

                # Second Trial
                data_eye_hc_second = data_hc[t[2]+902:t[2]+4204]
                #data_eye_hc_second = remove_outlier_IQR(data_eye_hc_second,'GazePointRightX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                #data_eye_hc_second  =  blink_nan(data_eye_hc_second['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                eight_deg_time_eye_second , eight_deg_pos_eye_second =  px_deg_eye(data_eye_hc_second,'right')
                
                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                eight_deg_second = pd.DataFrame({'x2_'+str(filename):eight_deg_time_eye_second,
                'y2_'+str(filename):eight_deg_pos_eye_second})


                df_subject = pd.concat([eight_deg_first,eight_deg_second],axis=1)

                frames.append(df_subject)
                processed_files_count += 1  # Increment the counter for each successfully processed file
                print('File '+ str(filename)+ ' is done.')
            else:
                print('File '+ str(os.path.basename(file_hc))+ ' is started.')
                # First Trial
                data_eye_hc_first = data_hc[t[0]+902:t[0]+4204] # first trial index
                #data_eye_hc_first  = remove_outlier_IQR(data_eye_hc_first,'GazePointLeftX (ADCSpx)')
                #data_eye_hc_first  = data_eye_hc_first.to_frame()
                #data_eye_hc_first  =  blink_nan(data_eye_hc_first['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
                #data_eye_hc_first  = data_eye_hc_first.to_frame()
                eight_deg_time_eye , eight_deg_pos_eye =  px_deg_eye(data_eye_hc_first,'left')

                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                eight_deg_first = pd.DataFrame({'x2_'+str(filename):eight_deg_time_eye,
                'y2_'+str(filename):eight_deg_pos_eye})


                # Second Trial
                data_eye_hc_second = data_hc[t[2]+902:t[2]+4204]
                #data_eye_hc_second= remove_outlier_IQR(data_eye_hc_second,'GazePointLeftX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                #data_eye_hc_second  =  blink_nan(data_eye_hc_second['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
                #data_eye_hc_second  = data_eye_hc_second.to_frame()
                eight_deg_time_eye_second , eight_deg_pos_eye_second =  px_deg_eye(data_eye_hc_second,'left')

                #Remove left and right
                filename = os.path.basename(file_hc)
                filename = filename.replace('_right','').replace('_left','')
                eight_deg_second = pd.DataFrame({'x2_'+str(filename):eight_deg_time_eye_second,
                'y2_'+str(filename):eight_deg_pos_eye_second})
                

                df_subject = pd.concat([eight_deg_first,eight_deg_second],axis=1)

                frames.append(df_subject)
                processed_files_count += 1  # Increment the counter for each successfully processed file
                print('File '+ str(filename)+ ' is done.')
        except Exception as e:
            print(f"Error processing file {os.path.basename(file_hc)}: {e}")
    print("Total number of processed files:", processed_files_count)  # Print the counter value after the loop

    eight_deg = pd.concat(frames,axis=1)
   
    eight_deg.to_csv('eight_degHC.csv',index=False)

In [17]:
EightCsvHC('8_dps.xlsx')

File PD009_right is started.
File PD009 is done.
File PDN015_right is started.
File PDN015 is done.
File PDN017_right is started.
File PDN017 is done.
File PD012_left is started.
File PD012 is done.
File PD004_left is started.
File PD004 is done.
File PD013_right is started.
File PD013 is done.
File PD002_left is started.
File PD002 is done.
File PD008_right is started.
File PD008 is done.
File PD011_right is started.
File PD011 is done.
File PD003_left is started.
File PD003 is done.
File PD026_right is started.
File PD026 is done.
File PD024_right is started.
File PD024 is done.
File PD020_right is started.
File PD020 is done.
File PDN018_right is started.
File PDN018 is done.
File PD022_right is started.
File PD022 is done.
File PD001_right is started.
File PD001 is done.
File PD006_left is started.
File PD006 is done.
File PD010_left is started.
File PD010 is done.
File PD007_right is started.
File PD007 is done.
File PD023_right is started.
File PD023 is done.
File PDN019_right is